In [1]:
import textract
import os
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
root = Path(".")

In [2]:
article_1 = pd.read_csv('articles1.csv')

In [3]:
news_content = article_1['content'][:500]

In [4]:
documents = list()

for file in news_content:
    ans = list()
    for i in range(len(file)):
        if file[i].isalnum():
            ans.append(file[i].lower())
        else:
            ans.append(' ')
    
    res = list()
    for i in range(len(ans) - 1):
        if ans[i] == ans[i + 1] and ans[i] == ' ':
            continue
        else:
            res.append(ans[i])
    
    if len(ans) > 0:
        res.append(ans[-1])
        documents.append(''.join(res))

In [5]:
def get_shingle_set(documents, k):
    shingles = set()
    shingle_doc_id = dict()

    for j in range(len(documents)):
        for i in range(len(documents[j]) - k + 1):
            cur_shingle = documents[j][i : i + k]

            shingles.add(cur_shingle)
            
            if shingle_doc_id.get(cur_shingle) == None:
                shingle_doc_id[cur_shingle] = [j]
            else:
                shingle_doc_id[cur_shingle].append(j)
    
    return (shingles, shingle_doc_id)

In [6]:
for k in range(2, 11):
    print(k, len(get_shingle_set(documents, k)[0]))

2 994
3 9223
4 47147
5 149523
6 358639
7 669031
8 1040926
9 1432704
10 1802365


In [7]:
K = 5
shingles, shingle_doc_id = get_shingle_set(documents, K)
shingles = sorted(list(shingles))

In [8]:
incident_matrix = np.zeros(shape=(len(shingles), len(documents)))
incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
shingle_id = dict()
for i in range(len(shingles)):
    shingle_id[shingles[i]] = i

for shingle, doc_ids in shingle_doc_id.items():
    for doc_id in doc_ids:
        incident_matrix[shingle_id[shingle]][doc_id] = 1

incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
my_path = root / "Pickled_files" / "Incident_Matrix"
dbfile = open(my_path, 'wb')
pickle.dump(incident_matrix, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingles"
dbfile = open(my_path, 'wb')
pickle.dump(shingles, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingle_id"
dbfile = open(my_path, 'wb')
pickle.dump(shingle_id, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "documents"
dbfile = open(my_path, 'wb')
pickle.dump(documents, dbfile) 
dbfile.close()

In [12]:
for i in range(len(documents)): 
    matrix = incident_matrix[0 : len(incident_matrix), i]
    print(np.count_nonzero(matrix == 1), matrix.shape[0])

3184 149523
11614 149523
7474 149523
7986 149523
2497 149523
720 149523
2401 149523
8476 149523
5525 149523
3122 149523
4143 149523
5832 149523
4995 149523
2477 149523
5665 149523
4889 149523
5362 149523
5117 149523
2375 149523
1330 149523
3268 149523
2740 149523
4569 149523
5731 149523
4848 149523
3510 149523
4576 149523
5883 149523
3243 149523
8068 149523
3805 149523
4271 149523
3237 149523
1768 149523
2110 149523
2972 149523
4930 149523
4025 149523
2662 149523
4469 149523
4323 149523
4564 149523
4896 149523
5278 149523
2279 149523
4683 149523
3260 149523
1620 149523
4517 149523
4021 149523
5737 149523
4621 149523
1598 149523
3052 149523
5137 149523
5900 149523
2782 149523
3921 149523
3771 149523
3360 149523
2687 149523
4187 149523
5022 149523
4749 149523
3898 149523
11594 149523
2301 149523
2651 149523
2522 149523
2747 149523
2805 149523
16274 149523
4044 149523
4012 149523
3233 149523
3383 149523
1490 149523
1721 149523
4370 149523
1409 149523
3585 149523
1496 149523
2531 149523
39